# 🚀 Google Colab Setup[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ogautier1980/sandbox-ml/blob/main/cours/04_classification_supervisee/04_exercices_solutions.ipynb)**Si vous exécutez ce notebook sur Google Colab**, exécutez la cellule suivante pour installer les dépendances.

In [ ]:
# Installation des dépendances (Google Colab uniquement)import sysIN_COLAB = 'google.colab' in sys.modulesif IN_COLAB:    print('📦 Installation des packages...')        # Packages ML de base    !pip install -q numpy pandas matplotlib seaborn scikit-learn        # Détection du chapitre et installation des dépendances spécifiques    notebook_name = '04_exercices.ipynb'  # Sera remplacé automatiquement        # Ch 06-08 : Deep Learning    if any(x in notebook_name for x in ['06_', '07_', '08_']):        !pip install -q torch torchvision torchaudio        # Ch 08 : NLP    if '08_' in notebook_name:        !pip install -q transformers datasets tokenizers        if 'rag' in notebook_name:            !pip install -q sentence-transformers faiss-cpu rank-bm25        # Ch 09 : Reinforcement Learning    if '09_' in notebook_name:        !pip install -q gymnasium[classic-control]        # Ch 04 : Boosting    if '04_' in notebook_name and 'boosting' in notebook_name:        !pip install -q xgboost lightgbm catboost        # Ch 05 : Clustering avancé    if '05_' in notebook_name:        !pip install -q umap-learn        # Ch 11 : Séries temporelles    if '11_' in notebook_name:        !pip install -q statsmodels prophet        # Ch 12 : Vision avancée    if '12_' in notebook_name:        !pip install -q ultralytics timm segmentation-models-pytorch        # Ch 13 : Recommandation    if '13_' in notebook_name:        !pip install -q scikit-surprise implicit        # Ch 14 : MLOps    if '14_' in notebook_name:        !pip install -q mlflow fastapi pydantic        print('✅ Installation terminée !')else:    print('ℹ️  Environnement local détecté, les packages sont déjà installés.')

# Chapitre 04 - Solutions de Classification SuperviséeCe notebook contient des exercices pratiques sur les algorithmes de classification.## Objectifs- Appliquer KNN, arbres, boosting et SVM- Comparer les performances- Optimiser les hyperparamètres- Diagnostiquer les problèmes

In [ ]:
import numpy as npimport pandas as pdimport matplotlib.pyplot as pltimport seaborn as snsfrom sklearn.data  # type: ignoresets import load_digits, fetch_covtypefrom sklearn.model_selection import train_test_split, cross_val_score, learning_curvefrom sklearn.preprocessing import StandardScalerfrom sklearn.neighbors import KNeighborsClassifierfrom sklearn.tree import DecisionTreeClassifierfrom sklearn.ensemble import RandomForestClassifierfrom sklearn.svm import SVCimport xgboost as xgbfrom sklearn.metrics import (    accuracy_score, precision_score, recall_score, f1_score,    confusion_matrix, classification_report, ConfusionMatrixDisplay)from time import timeimport warningswarnings.filterwarnings('ignore')sns.set_style('whitegrid')plt.rcParams['figure.figsize'] = (12, 6)

## Exercice 1 : Classification de Chiffres Manuscrits (Digits Dataset)**Objectif** : Classifier les chiffres manuscrits (0-9) avec différents algorithmes.**Consignes** :1. Charger le dataset Digits (8x8 images)2. Explorer les données et visualiser quelques exemples3. Entraîner KNN, Decision Tree, Random Forest, XGBoost4. Comparer les performances (accuracy, F1, temps)5. Analyser les erreurs avec la matrice de confusion

In [ ]:
# 1. Chargement des donnéesdigits = load_digits()X_digits = digits.data  # type: ignorey_digits = digits.target  # type: ignoreprint(f"Shape: {X_digits.shape}")print(f"Nombre de classes: {len(np.unique(y_digits))}")print(f"Distribution des classes:\n{np.bincount(y_digits)}")

In [ ]:
# 2. Visualisation d'exemplesfig, axes = plt.subplots(2, 10, figsize=(16, 4))axes = axes.ravel()for i in range(20):    axes[i].imshow(digits.images[i], cmap='gray')    axes[i].set_title(f'Label: {y_digits[i]}')    axes[i].axis('off')plt.tight_layout()plt.show()

In [ ]:
# 3. Préparation des donnéesX_train_dig, X_test_dig, y_train_dig, y_test_dig = train_test_split(    X_digits, y_digits, test_size=0.2, random_state=42, stratify=y_digits)# Standardisation pour KNNscaler_dig = StandardScaler()X_train_dig_scaled = scaler_dig.fit_transform(X_train_dig)X_test_dig_scaled = scaler_dig.transform(X_test_dig)print(f"Train set: {X_train_dig.shape}")print(f"Test set: {X_test_dig.shape}")

In [ ]:
# 4. Entraînement et comparaison des modèlesmodels_dig = {    'KNN (k=5)': (KNeighborsClassifier(n_neighbors=5), True),  # True = needs scaling    'Decision Tree': (DecisionTreeClassifier(max_depth=10, random_state=42), False),    'Random Forest': (RandomForestClassifier(n_estimators=100, random_state=42), False),    'XGBoost': (xgb.XGBClassifier(n_estimators=100, random_state=42, eval_metric='mlogloss'), False)}results_dig = []for name, (model, needs_scaling) in models_dig.items():    # Choisir les données appropriées    X_tr = X_train_dig_scaled if needs_scaling else X_train_dig    X_te = X_test_dig_scaled if needs_scaling else X_test_dig        # Entraînement    start = time()    model.fit(X_tr, y_train_dig)    train_time = time() - start        # Prédiction    start = time()    y_pred = model.predict(X_te)    pred_time = time() - start        # Métriques    results_dig.append({        'Model': name,        'Train Acc': model.score(X_tr, y_train_dig),        'Test Acc': accuracy_score(y_test_dig, y_pred),        'Precision': precision_score(y_test_dig, y_pred, average='weighted'),        'Recall': recall_score(y_test_dig, y_pred, average='weighted'),        'F1': f1_score(y_test_dig, y_pred, average='weighted'),        'Train Time': train_time,        'Pred Time': pred_time    })results_dig_df = pd.DataFrame(results_dig)print("Résultats sur Digits Dataset:")print(results_dig_df.to_string(index=False))

In [ ]:
# Visualisation des performancesfig, axes = plt.subplots(1, 3, figsize=(18, 5))# Accuracyaxes[0].barh(results_dig_df['Model'], results_dig_df['Test Acc'], alpha=0.7)axes[0].set_xlabel('Accuracy')axes[0].set_title('Test Accuracy')axes[0].set_xlim(0.8, 1.0)# F1 Scoreaxes[1].barh(results_dig_df['Model'], results_dig_df['F1'], alpha=0.7, color='orange')axes[1].set_xlabel('F1 Score')axes[1].set_title('F1 Score (Weighted)')axes[1].set_xlim(0.8, 1.0)# Train Timeaxes[2].barh(results_dig_df['Model'], results_dig_df['Train Time'], alpha=0.7, color='green')axes[2].set_xlabel('Temps (s)')axes[2].set_title('Temps d\'Entraînement')plt.tight_layout()plt.show()

In [ ]:
# 5. Analyse des erreurs (meilleur modèle)best_idx = results_dig_df['Test Acc'].idxmax()best_model_name = results_dig_df.loc[best_idx, 'Model']best_model, needs_scaling = models_dig[best_model_name]X_te = X_test_dig_scaled if needs_scaling else X_test_digy_pred_best = best_model.predict(X_te)print(f"Meilleur modèle: {best_model_name}")print(f"\nClassification Report:")print(classification_report(y_test_dig, y_pred_best))# Matrice de confusioncm = confusion_matrix(y_test_dig, y_pred_best)plt.figure(figsize=(10, 8))disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=digits.target  # type: ignore_names)disp.plot(cmap='Blues')plt.title(f'Matrice de Confusion - {best_model_name}')plt.show()# Visualiser quelques erreurserrors = y_test_dig != y_pred_bestX_errors = X_test_dig[errors]y_true_errors = y_test_dig[errors]y_pred_errors = y_pred_best[errors]print(f"\nNombre d'erreurs: {errors.sum()} / {len(y_test_dig)} ({100*errors.sum()/len(y_test_dig):.2f}%)")fig, axes = plt.subplots(2, 5, figsize=(14, 6))axes = axes.ravel()for i in range(min(10, len(X_errors))):    axes[i].imshow(X_errors[i].reshape(8, 8), cmap='gray')    axes[i].set_title(f'True: {y_true_errors[i]}\nPred: {y_pred_errors[i]}', color='red')    axes[i].axis('off')plt.suptitle('Exemples d\'Erreurs de Classification')plt.tight_layout()plt.show()

## Exercice 2 : Optimisation des Hyperparamètres**Objectif** : Optimiser un Random Forest et un SVM sur le dataset Digits.**Consignes** :1. Random Forest: Optimiser `n_estimators`, `max_depth`, `min_samples_split`2. SVM RBF: Optimiser `C` et `gamma`3. Comparer les performances avant/après optimisation4. Générer les learning curves

In [ ]:
# 1. Optimisation Random Forestfrom sklearn.model_selection import RandomizedSearchCVparam_dist_rf = {    'n_estimators': [50, 100, 200, 300],    'max_depth': [5, 10, 15, 20, None],    'min_samples_split': [2, 5, 10],    'min_samples_leaf': [1, 2, 4]}rf = RandomForestClassifier(random_state=42)rf_search = RandomizedSearchCV(    rf, param_dist_rf, n_iter=20, cv=5,     scoring='f1_weighted', random_state=42, n_jobs=-1, verbose=1)print("Optimisation Random Forest...")rf_search.fit(X_train_dig, y_train_dig)print(f"\nMeilleurs paramètres: {rf_search.best_params_}")print(f"Meilleur score F1 (CV): {rf_search.best_score_:.4f}")# Comparaison avant/aprèsrf_default = RandomForestClassifier(random_state=42)rf_default.fit(X_train_dig, y_train_dig)y_pred_default = rf_default.predict(X_test_dig)y_pred_optimized = rf_search.best_estimator_.predict(X_test_dig)print("\nComparaison Random Forest:")print(f"Défaut - Test Accuracy: {accuracy_score(y_test_dig, y_pred_default):.4f}")print(f"Optimisé - Test Accuracy: {accuracy_score(y_test_dig, y_pred_optimized):.4f}")

In [ ]:
# 2. Optimisation SVM RBFfrom sklearn.model_selection import GridSearchCVparam_grid_svm = {    'C': [0.1, 1, 10, 100],    'gamma': ['scale', 0.001, 0.01, 0.1, 1]}svm = SVC(kernel='rbf')svm_search = GridSearchCV(    svm, param_grid_svm, cv=5,     scoring='f1_weighted', n_jobs=-1, verbose=1)print("Optimisation SVM RBF...")svm_search.fit(X_train_dig_scaled, y_train_dig)print(f"\nMeilleurs paramètres: {svm_search.best_params_}")print(f"Meilleur score F1 (CV): {svm_search.best_score_:.4f}")# Comparaison avant/aprèssvm_default = SVC(kernel='rbf')svm_default.fit(X_train_dig_scaled, y_train_dig)y_pred_svm_default = svm_default.predict(X_test_dig_scaled)y_pred_svm_optimized = svm_search.best_estimator_.predict(X_test_dig_scaled)print("\nComparaison SVM:")print(f"Défaut - Test Accuracy: {accuracy_score(y_test_dig, y_pred_svm_default):.4f}")print(f"Optimisé - Test Accuracy: {accuracy_score(y_test_dig, y_pred_svm_optimized):.4f}")

In [ ]:
# 4. Learning Curvesmodels_lc = {    'RF Défaut': rf_default,    'RF Optimisé': rf_search.best_estimator_,    'SVM Défaut': svm_default,    'SVM Optimisé': svm_search.best_estimator_}fig, axes = plt.subplots(2, 2, figsize=(16, 12))axes = axes.ravel()for idx, (name, model) in enumerate(models_lc.items()):    # Choisir les données appropriées    if 'SVM' in name:        X_lc = X_train_dig_scaled    else:        X_lc = X_train_dig        train_sizes, train_scores, val_scores = learning_curve(        model, X_lc, y_train_dig,        train_sizes=np.linspace(0.1, 1.0, 10),        cv=5,        scoring='accuracy',        n_jobs=-1    )        train_mean = np.mean(train_scores, axis=1)    train_std = np.std(train_scores, axis=1)    val_mean = np.mean(val_scores, axis=1)    val_std = np.std(val_scores, axis=1)        axes[idx].plot(train_sizes, train_mean, 'o-', label='Train', linewidth=2)    axes[idx].fill_between(train_sizes, train_mean - train_std,                             train_mean + train_std, alpha=0.2)        axes[idx].plot(train_sizes, val_mean, 's-', label='Validation', linewidth=2)    axes[idx].fill_between(train_sizes, val_mean - val_std,                             val_mean + val_std, alpha=0.2)        axes[idx].set_xlabel('Taille du Training Set')    axes[idx].set_ylabel('Accuracy')    axes[idx].set_title(f'Learning Curve: {name}')    axes[idx].legend()    axes[idx].grid(True, alpha=0.3)plt.tight_layout()plt.show()

## Exercice 3 : Gestion du Déséquilibre des Classes**Objectif** : Traiter un dataset déséquilibré avec différentes techniques.**Consignes** :1. Créer un dataset synthétique déséquilibré (95% classe 0, 5% classe 1)2. Entraîner un modèle baseline (sans traitement)3. Utiliser class_weight='balanced'4. Sous-échantillonner la classe majoritaire5. Sur-échantillonner la classe minoritaire (SMOTE)6. Comparer les performances avec précision, rappel, F1

In [ ]:
# 1. Création d'un dataset déséquilibréfrom sklearn.data  # type: ignoresets import make_classificationX_imb, y_imb = make_classification(    n_samples=1000,    n_features=20,    n_informative=15,    n_redundant=5,    n_classes=2,    weights=[0.95, 0.05],  # 95% classe 0, 5% classe 1    random_state=42)print(f"Distribution des classes: {np.bincount(y_imb)}")print(f"Proportion classe 1: {100 * y_imb.sum() / len(y_imb):.2f}%")X_train_imb, X_test_imb, y_train_imb, y_test_imb = train_test_split(    X_imb, y_imb, test_size=0.3, random_state=42, stratify=y_imb)

In [ ]:
# 2. Baseline (sans traitement)rf_baseline = RandomForestClassifier(n_estimators=100, random_state=42)rf_baseline.fit(X_train_imb, y_train_imb)y_pred_baseline = rf_baseline.predict(X_test_imb)print("Baseline (sans traitement du déséquilibre):")print(classification_report(y_test_imb, y_pred_baseline, target_names=['Classe 0', 'Classe 1']))

In [ ]:
# 3. class_weight='balanced'rf_balanced = RandomForestClassifier(n_estimators=100, class_weight='balanced', random_state=42)rf_balanced.fit(X_train_imb, y_train_imb)y_pred_balanced = rf_balanced.predict(X_test_imb)print("Avec class_weight='balanced':")print(classification_report(y_test_imb, y_pred_balanced, target_names=['Classe 0', 'Classe 1']))

In [ ]:
# 4. Sous-échantillonnage de la classe majoritairefrom imblearn.under_sampling import RandomUnderSamplerrus = RandomUnderSampler(random_state=42)X_train_under, y_train_under = rus.fit_resample(X_train_imb, y_train_imb)print(f"Après sous-échantillonnage: {np.bincount(y_train_under)}")rf_under = RandomForestClassifier(n_estimators=100, random_state=42)rf_under.fit(X_train_under, y_train_under)y_pred_under = rf_under.predict(X_test_imb)print("\nAvec sous-échantillonnage:")print(classification_report(y_test_imb, y_pred_under, target_names=['Classe 0', 'Classe 1']))

In [ ]:
# 5. Sur-échantillonnage avec SMOTEfrom imblearn.over_sampling import SMOTEsmote = SMOTE(random_state=42)X_train_smote, y_train_smote = smote.fit_resample(X_train_imb, y_train_imb)print(f"Après SMOTE: {np.bincount(y_train_smote)}")rf_smote = RandomForestClassifier(n_estimators=100, random_state=42)rf_smote.fit(X_train_smote, y_train_smote)y_pred_smote = rf_smote.predict(X_test_imb)print("\nAvec SMOTE:")print(classification_report(y_test_imb, y_pred_smote, target_names=['Classe 0', 'Classe 1']))

In [ ]:
# 6. Comparaison globalestrategies = {    'Baseline': y_pred_baseline,    'Balanced Weights': y_pred_balanced,    'Under-sampling': y_pred_under,    'SMOTE': y_pred_smote}results_imb = []for name, y_pred in strategies.items():    results_imb.append({        'Strategy': name,        'Accuracy': accuracy_score(y_test_imb, y_pred),        'Precision (Class 1)': precision_score(y_test_imb, y_pred),        'Recall (Class 1)': recall_score(y_test_imb, y_pred),        'F1 (Class 1)': f1_score(y_test_imb, y_pred)    })results_imb_df = pd.DataFrame(results_imb)print("\nComparaison des Stratégies:")print(results_imb_df.to_string(index=False))# Visualisationfig, axes = plt.subplots(1, 3, figsize=(18, 5))metrics = ['Precision (Class 1)', 'Recall (Class 1)', 'F1 (Class 1)']for idx, metric in enumerate(metrics):    axes[idx].barh(results_imb_df['Strategy'], results_imb_df[metric], alpha=0.7)    axes[idx].set_xlabel(metric)    axes[idx].set_title(metric)    axes[idx].set_xlim(0, 1)plt.tight_layout()plt.show()print("\nConclusions:")print("- Baseline: Haute accuracy mais mauvais rappel pour classe minoritaire")print("- Balanced Weights: Compromis raisonnable")print("- Under-sampling: Perte d'information, peut dégrader les performances")print("- SMOTE: Souvent le meilleur compromis, génère des exemples synthétiques")

## Récapitulatif### Points clés abordés1. **Classification Multi-classe**   - Application sur dataset Digits (10 classes)   - Comparaison KNN, arbres, boosting   - Analyse des erreurs avec matrice de confusion2. **Optimisation des Hyperparamètres**   - GridSearchCV pour recherche exhaustive   - RandomizedSearchCV pour recherche aléatoire (plus rapide)   - Learning curves pour diagnostiquer biais/variance   - Impact de l'optimisation sur les performances3. **Déséquilibre des Classes**   - Problème fréquent en pratique (fraude, maladies rares, etc.)   - Accuracy peut être trompeuse   - Stratégies:     - `class_weight='balanced'`: Pénalise les erreurs sur classe minoritaire     - Under-sampling: Réduit classe majoritaire     - Over-sampling (SMOTE): Augmente classe minoritaire   - Métriques appropriées: Précision, Rappel, F1, AUC-ROC### Recommandations pratiques1. **Exploration préliminaire**   - Toujours vérifier la distribution des classes   - Visualiser quelques exemples   - Comprendre les features2. **Choix du modèle**   - Commencer simple (arbres, KNN)   - Essayer ensemble methods (RF, XGBoost)   - SVM si dataset de taille raisonnable3. **Optimisation**   - Utiliser validation croisée   - RandomizedSearchCV si grand espace de recherche   - Ne pas sur-optimiser sur le test set4. **Déséquilibre**   - Détecter le déséquilibre tôt   - Choisir métriques appropriées   - Tester plusieurs stratégies   - SMOTE souvent efficace